In [128]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [184]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [227]:
image_matrix = imread("cat.jpg")

In [228]:
def stretch(X, a, b):
    X_cp = X.copy()
    rows, cols = X_cp.shape[:2]

    new_rows, new_cols = int(rows * b), int(cols * a)
    stretched = np.zeros((new_rows, new_cols, 3), dtype=X_cp.dtype)

    transformation = np.array([[a, 0], [0,b]])
    transformation_inv = np.linalg.inv(transformation)

    for i in range(new_rows):
        for j in range(new_cols):
            j_old, i_old = transformation_inv.dot(np.array([j, i]))
            j_old, i_old = int(round(j_old)), int(round(i_old))
            stretched[i, j] = X_cp[i_old, j_old] if 0 <= j_old < cols and 0 <= i_old < rows else 0
    return stretched

In [229]:
def shear(X, a, b):
    X_cp = X.copy()
    rows, cols = X_cp.shape[:2]

    new_rows, new_cols = int(rows * b), int(cols * a)
    sheared = np.zeros((new_rows, new_cols, 3), dtype=X_cp.dtype)

    transformation = np.array([[1, a], [b,1]])
    transformation_inv = np.linalg.inv(transformation)

    for i in range(new_rows):
        for j in range(new_cols):
            j_old, i_old = transformation_inv.dot(np.array([j, i]))
            j_old, i_old = int(round(j_old)), int(round(i_old))
            sheared[i, j] = X_cp[i_old, j_old] if 0 <= j_old < cols and 0 <= i_old < rows else 0
    return sheared

In [241]:
def reflection(X, a, b):
    X_cp = X.copy()
    rows, cols = X_cp.shape[:2]

    new_rows, new_cols = int(rows * b), int(cols * a)
    reflected = np.zeros((new_rows, new_cols, 3), dtype=X_cp.dtype)

    denominator = (a ** 2 + b ** 2)
    transformation = (1/denominator)*np.array([[a**2 - b**2, 2*a*b],[2*a*b, b**2 - a**2]])
    transformation_inv = np.linalg.inv(transformation)
    for i in range(new_rows):
        for j in range(new_cols):
            j_old, i_old = transformation_inv.dot(np.array([j, i]))
            j_old, i_old = int(round(j_old)), int(round(i_old))
            reflected[i, j] = X_cp[i_old, j_old] if (0 <= j_old < cols and 0 <= i_old < rows) else 0
    return reflected

In [245]:
def rotation(X,theta):
    X_cp = X.copy()
    rows, cols = X_cp.shape[:2]
    rotated = np.zeros((rows, cols, 3), dtype=X_cp.dtype)
    cx, cy = cols / 2, rows / 2

    transformation = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    transformation_inv = np.linalg.inv(transformation)
    for i in range(rows):
        for j in range(cols):
            xshift, yshift = j - cx, i - cy
            j_old, i_old = transformation_inv.dot(np.array([xshift, yshift]))
            j_old, i_old = int(round(j_old)), int(round(i_old))
            rotated[i, j] = X_cp[i_old, j_old] if (0 <= j_old < cols and 0 <= i_old < rows) else 0
    return rotated